# Práctica 5 : Regresión lineal regularizada, sesgo y varianza

In [132]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import scipy.optimize as opt

## Regresión Lineal Regularizada

In [160]:
def cargaDatos(file = 'ex5data1.mat'):
    m = loadmat(file)
    x_train = m['X']
    y_train = m['y']
    x_test = m['Xtest']
    y_test = m['ytest']
    x_val = m['Xval']
    y_val = m['yval']
    train = [x_train, y_train]
    test = [x_test, y_test]
    val = [x_val, y_val]
    return train, test, val

In [156]:
def coste_reg(X, Y, T, Lambda):
    H = (np.dot(X,T) - Y)**2
    Sum = H.sum()
    m = len(X)
    coste = Sum/(2*m) + sum([T[i]*T[i] for i in range(1,len(T))])
    return coste

In [161]:
def gradiente_reg(X, Y, T, Lambda):
    TAux = np.zeros(T.shape)
    m = np.shape(X)[0]
    n = np.shape(X)[1]
    H = np.dot(X, T)
    Dif = (H - Y)
    for j in range(n):
        Dif_j = np.dot(Dif , X[:, j])
        TAux[j] = (1 / m) * Dif_j.sum() 
        if j != 0:
            TAux[j] = TAux[j] + ((Lambda/m)*T[j] )
    return TAux

In [166]:
def main1():
    tr, te, vl = cargaDatos()
    X = tr[0]
    Y = tr[1]
    m = X.shape[0]
    X = np.hstack([np.ones( [m, 1 ] ) , X ] )
    T = np.ones(X.shape[1])
    grad = gradiente(X,Y,T,1)
    c = coste_reg(X,Y,T,1)
    print(c)
    print(grad)

In [167]:
main1()

7280.796956475271
[ -183.63618809 10810.47091324]


__Creo que el gradiente da distinto porque estoy probando con ejemplos que no son__

In [134]:
def preparaFunciones(Lambda):
    c = lambda Theta,X,Y : coste_reg(X,Y,Theta,Lambda)
    gr = lambda Theta,X,Y : gradiente_reg(X,Y,Theta,Lambda)
    
    return (c,gr)

In [150]:
def optimizar(X,Y,Lambda):
    coste , gradiente = preparaFunciones(Lambda)
    T = np.zeros(X.shape[1])
    result = opt.fmin_tnc(func = coste, x0 = T, fprime = gradiente, args = (X, Y))
    return result[0]

In [151]:
def main2():
    tr, te, vl = cargaDatos()
    X = tr[0]
    Y = tr[1]
    m = X.shape[0]
    X = np.hstack([np.ones( [m, 1 ] ) , X ] )
    T = np.ones(X.shape[1])
    
    r = optimizar(X,Y,0)
    print(r)

In [152]:
main2()

[1.12175893e+01 1.02339829e-13]
